In [1]:
import requests
from bs4 import BeautifulSoup as bs

In [2]:
import pandas as pd
import re

In [3]:
link1 = 'https://en.wikipedia.org/wiki/List_of_schools_in_Nigeria'

In [4]:
req = requests.get(link1)
req

<Response [200]>

In [5]:
soup = bs(req.text, 'html.parser')

In [6]:
school_all = soup.find_all(class_ = 'mw-body-content')

In [7]:
def get_col(soup_,text):
  try:
    for row in soup_.select('table.infobox.vcard tbody tr'):
      if len(row.select('.infobox-label')) > 0 and row.select('.infobox-label')[0].getText(strip = True).lower() == text :
        result = row.select('.infobox .infobox-data')[0].getText()
        return result
    else:
      return ''

  except ValueError:
    return ''

In [8]:
def geo1(soup_):
  try:
    return soup_.select('.geo-dec')[0].getText().strip()
  except:
    return ''

In [9]:
web=[]
web_soup=[]
for school in school_all[0].select('h2'):
  for x in school.find_next('ul'):
    # print(x)
    if x != '\n' and x.find('a').get('href').find('Portal') < 0 :
        web.append('https://en.wikipedia.org' + x.find('a').get('href'))
for i in web:
  web_soup.append(bs(requests.get(i).content,'html.parser'))

In [10]:
get_these = ['type', 'motto', 'established', 'website', 'principal']
sch_data = {'state':[],
            'school_name':[],
            'location':[],
            }
for school in school_all[0].select('h2'):
  for x in school.find_next('ul'):
    if len(x.getText().strip())>1 and (school.getText()[:-6].lower().find('state') or school.getText()[:-6].lower().find('federal')):
      # print((school.getText()[:-6].lower()or school.getText()[:-6].lower().find('federal')))
      sch_data['state'].append(school.getText()[:-6])
      sch_data['school_name'].append(x.getText(strip = True).split(',')[0])
      sch_data['location'].append(x.getText(strip = True))
for text in get_these:
  sch_data[text] = []
sch_data['geo1'] = []
for soup in web_soup:
  sch_data['geo1'].append(geo1(soup))
  for text in get_these:
    # sch_data[text]
    sch_data[text].append(get_col(soup,text))


In [11]:
for k in sch_data.keys():
  if len(sch_data[k]) == 90:
    sch_data[k] = sch_data[k][:87]

In [12]:
df = pd.DataFrame(sch_data)

def sep_details(text):
  if text.find(',') == -1 :
    return text.split()[-1]
  else:
    return ', '.join(text.split(',')[1:])
df

,state,school_name,location,type,motto,established,website,principal,geo1
0,Abia State,Government College Umuahia,Government College Umuahia,Secondary school,In Unum Luceant(May We Shine as One),1929,gcu.sch.ng,,5.499367°N 7.538167°E
1,Abia State,Ngwa High School,"Ngwa High School,Aba",Secondary school,"""Dives Intra""(""Rich Within"")","September 24, 1954; 69 years ago (September 24...",,Felix O. Erondu,
2,Akwa Ibom State,Federal Government College,"Federal Government College, Ikot Ekpene,Ikot E...",Government funded,Pro Unitate,1973,fgcikotekpene.sch.ng,,
3,Akwa Ibom State,Holy Family College,"Holy Family College,Abak",Catholic Mission School (Private),Eveteribus Veritas,1942,Official website,Rev. Fr.(Dr)Columbus Archibong,
4,Akwa Ibom State,Lutheran High School,"Lutheran High School, Obot Idim,Ibesikpo Asutan",,In Deo Stamus,,,Ifiok Umanah,4.584356°N 7.553148°E
...,...,...,...,...,...,...,...,...,...
82,Oyo State,Wesley College,"Wesley College, Ibadan","State government-controlled, Co-educational",(Yoruba: Bí ẹni tí ńse ìráńsẹ́)(Learning to Se...,5 November 1905,,Mr A. A Osunbumi 1988 - 1991\nMr A.A Omotoso N...,
83,Rivers State,Nigerian Turkish International Colleges(locati...,Nigerian Turkish International Colleges(locati...,,,,,,
84,Rivers State,Command Secondary Schools(locations across the...,Command Secondary Schools(locations across the...,,Strength through Knowledge (formerly 'Discipli...,,www.dcss.sch.ng,,
85,Several locations,Nigerian Turkish International Colleges(locati...,Nigerian Turkish International Colleges(locati...,,,,,,


In [13]:
df['location'] = df['location'].apply(lambda x: sep_details(x))

In [14]:
df1 = df.iloc[:83].copy()
df1.head()

,state,school_name,location,type,motto,established,website,principal,geo1
0,Abia State,Government College Umuahia,Umuahia,Secondary school,In Unum Luceant(May We Shine as One),1929,gcu.sch.ng,,5.499367°N 7.538167°E
1,Abia State,Ngwa High School,Aba,Secondary school,"""Dives Intra""(""Rich Within"")","September 24, 1954; 69 years ago (September 24...",,Felix O. Erondu,
2,Akwa Ibom State,Federal Government College,"Ikot Ekpene, Ikot Ekpene",Government funded,Pro Unitate,1973,fgcikotekpene.sch.ng,,
3,Akwa Ibom State,Holy Family College,Abak,Catholic Mission School (Private),Eveteribus Veritas,1942,Official website,Rev. Fr.(Dr)Columbus Archibong,
4,Akwa Ibom State,Lutheran High School,"Obot Idim, Ibesikpo Asutan",,In Deo Stamus,,,Ifiok Umanah,4.584356°N 7.553148°E


In [15]:
import re

In [16]:
df1.head()

,state,school_name,location,type,motto,established,website,principal,geo1
0,Abia State,Government College Umuahia,Umuahia,Secondary school,In Unum Luceant(May We Shine as One),1929,gcu.sch.ng,,5.499367°N 7.538167°E
1,Abia State,Ngwa High School,Aba,Secondary school,"""Dives Intra""(""Rich Within"")","September 24, 1954; 69 years ago (September 24...",,Felix O. Erondu,
2,Akwa Ibom State,Federal Government College,"Ikot Ekpene, Ikot Ekpene",Government funded,Pro Unitate,1973,fgcikotekpene.sch.ng,,
3,Akwa Ibom State,Holy Family College,Abak,Catholic Mission School (Private),Eveteribus Veritas,1942,Official website,Rev. Fr.(Dr)Columbus Archibong,
4,Akwa Ibom State,Lutheran High School,"Obot Idim, Ibesikpo Asutan",,In Deo Stamus,,,Ifiok Umanah,4.584356°N 7.553148°E


In [17]:
df1['established'] = df1['established'].apply(lambda x: re.search(r'\b\d{4}\b',x)[0] if re.search(r'\b\d{4}\b',x) else '')

In [18]:
df1['established'] = pd.to_numeric(df1['established'], downcast = 'integer')

In [20]:
df1['website'] = df1['website'].apply(lambda x :  re.findall(r'(?:https://)?(?:www\.)?[a-zA-Z0-9-]+\.[a-zA-Z]{2,}\.[a-zA_Z]+',x)[0] \
                     if re.findall(r'(?:https://)?(?:www\.)?[a-zA-Z0-9-]+\.[a-zA-Z]{2,}\.[a-zA_Z]+',x) else '')

In [21]:
df1['geo1'] = df1['geo1'].apply(lambda x: tuple(x.split()) if x.split() else '')

In [22]:
df1['principal'] = df1['principal'].apply(lambda x : x.splitlines()[-1] if x.splitlines() else x)

In [23]:
df1['principal'] = df1['principal'].apply(lambda x: re.findall(r"\D+",x)[0] if re.findall(r"\D",x) else '')
# df1

In [24]:
df1['type'] = df['type'].copy()

In [25]:
df1['type'] = df1['type'].apply(lambda x: 'Public' if re.match(r"(.*\b(?:Secondary|Government|Voluntary|Military|Coeducational|Public|State)\b.*)",x, re.IGNORECASE) else 'Private')

In [26]:
df1.head()

,state,school_name,location,type,motto,established,website,principal,geo1
0,Abia State,Government College Umuahia,Umuahia,Public,In Unum Luceant(May We Shine as One),1929.0,gcu.sch.ng,,"(5.499367°N, 7.538167°E)"
1,Abia State,Ngwa High School,Aba,Public,"""Dives Intra""(""Rich Within"")",1954.0,,Felix O. Erondu,
2,Akwa Ibom State,Federal Government College,"Ikot Ekpene, Ikot Ekpene",Public,Pro Unitate,1973.0,fgcikotekpene.sch.ng,,
3,Akwa Ibom State,Holy Family College,Abak,Private,Eveteribus Veritas,1942.0,,Rev. Fr.(Dr)Columbus Archibong,
4,Akwa Ibom State,Lutheran High School,"Obot Idim, Ibesikpo Asutan",Private,In Deo Stamus,NaN,,Ifiok Umanah,"(4.584356°N, 7.553148°E)"
